In [1]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals.joblib import Memory

from nltk.tokenize import RegexpTokenizer
import nltk
import pandas as pd
from nltk.corpus import stopwords
import FEATURE_SELECTOR_v4
import datetime
import json



from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Marius\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marius\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Marius\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def set_filter_params(var_word_n_grams=(True, 'absolute', [5000, 500, 500]),
                     var_char_n_grams=(True, 'absolute', [500, 500, 500, 500]),
                     var_pos_n_grams=(True, 'absolute', [500, 500, 500]),
                     freq_word_n_grams=(True, 'absolute', [2000, 2000, 2000]),
                     freq_char_n_grams=(True, 'absolute', [2000, 2000, 2000, 2000]),
                     freq_pos_n_grams=(True, 'absolute', [2000, 2000, 2000])):

    return {
        # Filter by variance: (flag, Type_of_filter, [thresholds for each n in
        # word_n_grams])
        'var_word_n_grams': var_word_n_grams,      
        # Filter by variance: (flag, Type_of_filter, [thresholds for each n in
        # char_n_grams])
        'var_char_n_grams':var_char_n_grams,   
        # Filter by variance: (flag, Type_of_filter, [thresholds for each n in
        # pos_n_grams])
        'var_pos_n_grams':var_pos_n_grams,       
        # Use the most common features: (flag, Type_of_filter, [thresholds for each n
        # in word_n_grams])
        'freq_word_n_grams':freq_word_n_grams,
        # Use the most common features: (flag, Type_of_filter, [thresholds for each n
        # in char_n_grams])
        'freq_char_n_grams':freq_char_n_grams,           
        # Use the most common features: (flag, Type_of_filter, [thresholds for each n
        # in pos_n_grams])
        'freq_pos_n_grams':freq_pos_n_grams
    }

In [3]:
def set_features_to_calc(char_n_grams= [1,2,3,4],        
    word_n_grams = [1,2,3],                                          
    pos_n_grams=[1,2,3],                      
    avg_sent_len=True,                         
    avg_word_len=True,                         
    token_per_sent=True,                      
    vocabulary_richness=True):

    return{
        # list with n for char_n_grams
        'char_n_grams': char_n_grams,        
        # list with n for word_n_grams
        'word_n_grams': word_n_grams,                                          
        # list with n for pos_n_grams
        'pos_n_grams': pos_n_grams,                      
        # flag if average sentence length should be calculated
        'avg_sent_len':avg_sent_len,                         
        # flag if average token length should be calculated
        'avg_word_len':avg_word_len,                         
        # flag if token per sentences should be calculated
        'token_per_sent':avg_word_len,                      
        # flag if vocabulary_richness should be calculated
        'vocabulary_richness':vocabulary_richness,                 
    }

In [4]:
def set_token_params_1 (
    word_tokenizer = RegexpTokenizer(r'\w+'),      
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle'), 
    get_tokens = 'sentence',                                                    
    word_strain = 'stem',
    filter_length = ('long', 0),                                                
    handle_stopwords = '',                                                      
    get_sentences = False,                                                      
    punctuation = "[,;.!—]",                                                   
    lower_stop_words = set(stopwords.words('english')),                         
    uncapitalized = True):
    
    
    return{
    # Word tokenizer
    'word_tokenizer':RegexpTokenizer(r'\w+'),  
    # Sentence tokenizer    
    'sent_tokenizer':nltk.data.load('tokenizers/punkt/english.pickle'), 
    # Group tokens: 'text'|'sentence'
    'get_tokens':'sentence',                                                    
    # 'lemma'|'stem'
    'word_strain':'stem',
    # Get tokens with length >= <int> or <= <int>: ('long',<int>)|('short',<int>)
    'filter_length':('long', 0),                                                
    # 'get'|'remove'
    'handle_stopwords':'',                                                      
    # Sentences tokens
    'get_sentences':False,                                                      
    # Remove punctuation
    'punctuation':"[,;.!—]",                                                   
    # Stopwords
    'lower_stop_words':set(stopwords.words('english')),                         
    # Original or uncapitalized stemms/lemmas
    'uncapitalized' : True                                                      
    }

In [5]:
#Path to data
path_data='.//datasets//constructed_2.csv'

df_texts=pd.read_csv(path_data,sep=',',header=0, index_col=0)
serie_texts=df_texts.text

df_texts.text.size

11039

In [6]:
token_params_1 = set_token_params_1 ()
features_to_calc=set_features_to_calc()
filter_params=set_filter_params(var_word_n_grams=(False,),
                     var_char_n_grams=(False, 'absolute', []),
                     var_pos_n_grams=(False, ),
                     freq_word_n_grams=(False, ),
                     freq_char_n_grams=(False,),
                     freq_pos_n_grams=(False,))
print(token_params_1)
print(features_to_calc)
print(filter_params)

{'word_tokenizer': RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>), 'sent_tokenizer': <nltk.tokenize.punkt.PunktSentenceTokenizer object at 0x00000268A9AAC2B0>, 'get_tokens': 'sentence', 'word_strain': 'stem', 'filter_length': ('long', 0), 'handle_stopwords': '', 'get_sentences': False, 'punctuation': '[,;.!—]', 'lower_stop_words': {'any', "needn't", 'd', 'to', 'below', "didn't", 'couldn', 'mightn', 'yourself', 'were', 'can', 'ma', "wasn't", 'an', 'herself', "you'd", 'the', 'yours', 'does', 'we', 'what', 'didn', 'after', "don't", 'so', 'now', "aren't", 'hers', 'her', "shan't", 'had', 'weren', 'once', "you're", 'wasn', 'as', 'by', 'shan', 'or', 'their', 'under', "it's", 'doing', 'be', 's', 'will', "hasn't", 'and', 'with', 'me', "that'll", 'won', 'all', 'too', 'it', 'have', 'than', 'my', "weren't", 'himself', 'theirs', 'until', 'isn', 'she', 'why', "mustn't", 'while', 'you', 'don', 'up', 'against', 'has', 'll', "haven't", 'd

In [8]:
# Export Path
path_selected_features='.//Features//test_features{0}.txt'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))
print(path_selected_features)

.//Features//test_features2019_01_03_14_14.txt


In [9]:
selected_features = FEATURE_SELECTOR_v4.select_features(
    # Contents
    serie_texts=serie_texts,                          
    # Dictionary with the features which should be calculated
    features_to_calc=features_to_calc, 
    # Dictionary with the parameters for tokenizing the contents
    token_params_1=token_params_1,              
    # Dictionary with the parameters for filtering the n_grams
    filter_params=filter_params,
    # Threshold for tokens (only token which occure in more documents than in 20%)
    cv_min_df = 0.1,                           
    # Flag if the features should be selected or extracted
    flag_extract_features=False,
    # Path to already selected features       
    path_to_features=path_selected_features,
    # TFIDF / Text_length
    normalization_type='')

KeyboardInterrupt: 

In [11]:
# Save the selected features
with open(path_selected_features, 'w') as f:
    json.dump(selected_features, f)

In [12]:
for key in selected_features.keys():
    print(key,selected_features[key])

word_n_grams_1 ['a', 'abandon', 'abl', 'about', 'accept', 'across', 'act', 'action', 'actual', 'after', 'again', 'against', 'age', 'agre', 'all', 'allow', 'almost', 'alon', 'along', 'alreadi', 'also', 'although', 'american', 'among', 'an', 'and', 'ani', 'anoth', 'appar', 'appear', 'are', 'armi', 'around', 'arriv', 'as', 'ask', 'assist', 'at', 'attack', 'attempt', 'author', 'away', 'back', 'base', 'battl', 'be', 'beauti', 'becaus', 'becom', 'been', 'befor', 'begin', 'behind', 'believ', 'best', 'between', 'black', 'bodi', 'book', 'both', 'boy', 'break', 'bring', 'brother', 'brought', 'build', 'but', 'by', 'call', 'can', 'cannot', 'captur', 'care', 'carri', 'case', 'caus', 'chang', 'charact', 'child', 'children', 'citi', 'claim', 'close', 'come', 'command', 'commit', 'commun', 'complet', 'concern', 'confront', 'consid', 'contact', 'continu', 'control', 'convinc', 'could', 'countri', 'creat', 'danger', 'dark', 'daughter', 'day', 'dead', 'deal', 'death', 'decid', 'describ', 'despit', 'destr

In [11]:
# Export path calculated features
PATH_CALC_FEATURES = './Features/test_calc_features{0}.txt'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))

In [14]:
# Extract features
extracted_features = FEATURE_SELECTOR_v4.select_features(
    # Contents
    serie_texts=serie_texts,                          
    # Dictionary with the features which should be calculated
    features_to_calc=features_to_calc, 
    # Dictionary with the parameters for tokenizing the contents
    token_params_1=token_params_1,              
    # Dictionary with the parameters for filtering the n_grams
    filter_params=filter_params,
    # Threshold for tokens
    cv_min_df = 0,                           
    # Flag if the features should be selected or extracted
    flag_extract_features=True,
    # Path to already selected features       
    path_to_features=path_selected_features,    
    # TFIDF / Text_length
    normalization_type='')

In [14]:
# Save the calculated features
extracted_features.to_csv(path_or_buf=PATH_CALC_FEATURES, sep=',', header=True, index=True)

NameError: name 'extracted_features' is not defined

In [12]:
extracted_features=pd.read_csv(filepath_or_buffer='./Features/test_calc_features2018_12_25_23_09.txt', sep=',', header=0, index_col=0)

In [13]:
# REVIEW: we shouldn't get NaN values in the first place
# Fill NaN values in features
extracted_features.fillna(0, inplace=True)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(extracted_features, df_texts.label, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(extracted_features, df_texts.label, test_size=0.33, random_state=42)

### Compare Classifiers

In [16]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

model0= SVC(gamma='auto')
model0.fit(X_train, y_train)
print('svm_clf',model0.score(X_test, y_test))

model1 = MultinomialNB(alpha=0.001)
model1.fit( X_train, y_train )
print('MultinomialNB',model1.score(X_test, y_test))

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( X_train, y_train )
print('SGDClassifier',model2.score(X_test, y_test))

model3 = RandomForestClassifier()
model3.fit( X_train, y_train )
print('RandomForestClassifier',model3.score(X_test, y_test))

model4 = GradientBoostingClassifier()
model4.fit( X_train, y_train )
print('GradientBoostingClassifier',model4.score(X_test, y_test))

model5=MLPClassifier(hidden_layer_sizes=(200,))
model5.fit( X_train, y_train )
print('MLPClassifier',model5.score(X_test, y_test))

svm_clf 0.5871534449629426
MultinomialNB 0.643425748009882


C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier 0.47845182541861103


C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier 0.7106780126269558
GradientBoostingClassifier 0.7878122426571507


C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier 0.6900905846829536


### PCA

In [17]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
extracted_features_scaled=pd.DataFrame(scaler.fit_transform(extracted_features))
extracted_features_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,...,10628,10629,10630,10631,10632,10633,10634,10635,10636,10637
count,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,...,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04,1.103900e+04
mean,-6.076206e-16,2.574663e-16,-2.368690e-16,-5.767246e-16,-2.111224e-16,1.609165e-19,8.882589e-17,5.149327e-18,4.119461e-17,-3.572345e-17,...,-1.930998e-17,-1.705715e-17,-7.209058e-17,-4.344745e-17,6.436659e-19,5.278060e-17,-1.158599e-17,4.376928e-17,1.802264e-17,5.149327e-18
std,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,...,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00
min,-2.920921e+00,-5.967601e+00,-3.348115e+00,-2.646035e+00,-2.035737e+00,-2.227685e-01,-3.032335e-01,-5.100637e-01,-2.794832e-01,-1.955677e-01,...,-2.312851e-01,-2.344928e-01,-2.471872e-01,-2.096703e-01,-3.080339e-01,-2.955312e-01,-3.993123e-01,-3.121379e-01,-2.867378e-01,-4.192929e-01
25%,-5.679949e-01,-5.665708e-01,-5.667396e-01,-7.546716e-01,-5.833672e-01,-2.227685e-01,-3.032335e-01,-5.100637e-01,-2.794832e-01,-1.955677e-01,...,-2.312851e-01,-2.344928e-01,-2.471872e-01,-2.096703e-01,-3.080339e-01,-2.955312e-01,-3.993123e-01,-3.121379e-01,-2.867378e-01,-4.192929e-01
50%,-1.434806e-01,-7.645394e-02,-1.409876e-01,-1.223148e-01,-1.385598e-01,-2.227685e-01,-3.032335e-01,-5.100637e-01,-2.794832e-01,-1.955677e-01,...,-2.312851e-01,-2.344928e-01,-2.471872e-01,-2.096703e-01,-3.080339e-01,-2.955312e-01,-3.993123e-01,-3.121379e-01,-2.867378e-01,-4.192929e-01
75%,3.840530e-01,4.650777e-01,3.865312e-01,6.456352e-01,3.973758e-01,-2.227685e-01,-3.032335e-01,1.978228e-01,-2.794832e-01,-1.955677e-01,...,-2.312851e-01,-2.344928e-01,-2.471872e-01,-2.096703e-01,-3.080339e-01,-2.955312e-01,-2.201768e-02,-3.121379e-01,-2.867378e-01,1.468682e-01
max,2.976557e+01,2.732082e+01,2.652905e+01,3.410068e+00,1.300829e+01,2.282587e+01,2.094566e+01,2.293868e+01,3.232037e+01,5.374518e+01,...,4.299104e+01,2.387181e+01,4.020338e+01,3.181282e+01,3.435760e+01,3.152126e+01,2.165790e+01,2.340699e+01,2.861880e+01,4.389225e+01


In [18]:
# PCA
from sklearn.decomposition import PCA

pca=PCA(n_components=10, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
extracted_features_pca=pca.fit_transform(extracted_features)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(extracted_features_pca, df_texts.label, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(extracted_features, df_texts.label, test_size=0.33, random_state=42)

### Further Classifiers

In [22]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

model0= SVC(gamma='auto')
model0.fit(X_train, y_train)
print('svm_clf',model0.score(X_test, y_test))

#model1 = MultinomialNB(alpha=0.001)
#model1.fit( X_train, y_train )
#print('MultinomialNB',model1.score(X_test, y_test))

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( X_train, y_train )
print('SGDClassifier',model2.score(X_test, y_test))

model3 = RandomForestClassifier()
model3.fit( X_train, y_train )
print('RandomForestClassifier',model3.score(X_test, y_test))

model4 = GradientBoostingClassifier()
model4.fit( X_train, y_train )
print('GradientBoostingClassifier',model4.score(X_test, y_test))

model5=MLPClassifier(hidden_layer_sizes=(200,))
model5.fit( X_train, y_train )
print('MLPClassifier',model5.score(X_test, y_test))

svm_clf 0.6272303046939336
SGDClassifier 0.5237441668954159


C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier 0.6291517979687071
GradientBoostingClassifier 0.6722481471314851
MLPClassifier 0.6560527038155366


## SVM with the first 4 columns

In [23]:
X_train, X_test, y_train, y_test= train_test_split(extracted_features.iloc[:,:4], df_texts.label, test_size=0.33, random_state=42)

In [24]:
svm_clf = SVC(gamma='auto')
svm_clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
print(svm_clf.score(X_test, y_test))

0.6384847653033214


### Further Classifiers

In [26]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

model1 = MultinomialNB(alpha=0.001)
model1.fit( X_train, y_train )
print('MultinomialNB',model1.score(X_test, y_test))

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( X_train, y_train )
print('SGDClassifier',model2.score(X_test, y_test))

model3 = RandomForestClassifier()
model3.fit( X_train, y_train )
print('RandomForestClassifier',model3.score(X_test, y_test))

model4 = GradientBoostingClassifier()
model4.fit( X_train, y_train )
print('GradientBoostingClassifier',model4.score(X_test, y_test))

model5=MLPClassifier(hidden_layer_sizes=(100,))
model5.fit( X_train, y_train )
print('MLPClassifier',model5.score(X_test, y_test))

MultinomialNB 0.5940159209442767
SGDClassifier 0.47817732637935767


C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Marius\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier 0.6148778479275323
GradientBoostingClassifier 0.674444139445512
MLPClassifier 0.6648366730716443


## Sequences

In [27]:
def sequences(s):
    words = s.str.split()
    words = pd.DataFrame(words.apply(np.array_split, indices_or_sections=4).tolist())
    return words.applymap(lambda s: " ".join(s) )

## SVM with sequences and all features

In [28]:
text_seqs = sequences(serie_texts)

In [29]:
print(type(text_seqs))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
df_features_seq=pd.DataFrame()
for index, col in enumerate(text_seqs.columns):
    print(index)
    df_features_seq=pd.concat((df_features_seq,FEATURE_SELECTOR_v4.select_features(serie_texts=text_seqs.iloc[:,index],features_to_calc=features_to_calc,token_params_1=token_params_1,filter_params=filter_params,path_to_features='./Features/test_features2018_12_25_21_31.txt',flag_extract_features=True,cv_min_df=0,normalization_type='')),axis=1,ignore_index=True) 

In [ ]:
print(df_features_seq.info())
print()

In [ ]:
# REVIEW: we shouldn't get NaN values in the first place
# Fill NaN values in features
df_features_seq.fillna(0, inplace=True)
df_features_seq.replace([np.inf, -np.inf], np.float64(0),inplace=True)

In [ ]:
np.where(np.any(np.isinf(df_features_seq.values), axis=0))[0]

In [ ]:
df_features_seq.loc[np.any(np.isinf(df_features_seq.values), axis=0),np.any(np.isinf(df_features_seq.values), axis=1)]


In [ ]:
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(df_features_seq, df_texts.label, test_size=0.33, random_state=42)


In [ ]:
svm_clf_seq = SVC(gamma='auto')
svm_clf_seq.fit(X_train_seq, y_train_seq)

In [ ]:
print(svm_clf_seq.score(X_test_seq, y_test_seq))

### Further Classifiers

In [ ]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

model1_seq = MultinomialNB(alpha=0.001)
model1_seq.fit( X_train_seq, y_train_seq )
print('MultinomialNB',model1_seq.score(X_test_seq, y_test_seq))

model2_seq = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2_seq.fit( X_train_seq, y_train_seq )
print('SGDClassifier',model2_seq.score(X_test_seq, y_test_seq))

model3_seq = RandomForestClassifier()
model3_seq.fit( X_train_seq, y_train_seq )
print('RandomForestClassifier',model3_seq.score(X_test_seq, y_test_seq))

model4_seq = GradientBoostingClassifier()
model4_seq.fit( X_train_seq, y_train_seq )
print('GradientBoostingClassifier',model4_seq.score(X_test_seq, y_test_seq))

model5_seq=MLPClassifier(hidden_layer_sizes=(100,))
model5_seq.fit( X_train_seq, y_train_seq )
print('MLPClassifier',model5_seq.score(X_test_seq, y_test_seq))

## SVM with sequences and 4 features

In [52]:
print(type(text_seqs))

<class 'pandas.core.frame.DataFrame'>


In [53]:
df_features_seq=pd.DataFrame()
for index, col in enumerate(text_seqs.columns):
    df_features_seq=pd.concat((df_features_seq,FEATURE_SELECTOR_v4.select_features(serie_texts=text_seqs.iloc[:,index],features_to_calc=features_to_calc,token_params_1=token_params_1,filter_params=filter_params,path_to_features=path_selected_features,flag_extract_features=True,cv_min_df=0,normalization_type=''))[0:4],axis=1,ignore_index=True) 

MemoryError: 

In [ ]:
print(df_features_seq.info())
print()

In [ ]:
# REVIEW: we shouldn't get NaN values in the first place
# Fill NaN values in features
df_features_seq.fillna(0, inplace=True)
df_features_seq.replace([np.inf, -np.inf], np.float64(0),inplace=True)

In [ ]:
np.where(np.any(np.isinf(df_features_seq.values), axis=0))[0]

In [ ]:
df_features_seq.loc[np.any(np.isinf(df_features_seq.values), axis=0),np.any(np.isinf(df_features_seq.values), axis=1)]


In [ ]:
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(df_features_seq, df_texts.label, test_size=0.33, random_state=42)


In [ ]:
svm_clf_seq = SVC(gamma='auto')
svm_clf_seq.fit(X_train_seq, y_train_seq)

In [ ]:
print(svm_clf_seq.score(X_test_seq, y_test_seq))

### Further Classifiers

In [ ]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

model1_seq = MultinomialNB(alpha=0.001)
model1_seq.fit( X_train_seq, y_train_seq )
print('MultinomialNB',model1_seq.score(X_test_seq, y_test_seq))

model2_seq = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2_seq.fit( X_train_seq, y_train_seq )
print('SGDClassifier',model2_seq.score(X_test_seq, y_test_seq))

model3_seq = RandomForestClassifier()
model3_seq.fit( X_train_seq, y_train_seq )
print('RandomForestClassifier',model3_seq.score(X_test_seq, y_test_seq))

model4_seq = GradientBoostingClassifier()
model4_seq.fit( X_train_seq, y_train_seq )
print('GradientBoostingClassifier',model4_seq.score(X_test_seq, y_test_seq))

model5_seq=MLPClassifier(hidden_layer_sizes=(100,))
model5_seq.fit( X_train_seq, y_train_seq )
print('MLPClassifier',model5_seq.score(X_test_seq, y_test_seq))

## Import Keras modules

In [74]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Keras simple dense NN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential()
 
model.add(Dense(units=500, activation='relu', input_dim=len(df_features_seq.columns))
model.add(Dense(units=1, activation='softmax'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train_onehot[:-100], y_train[:-100], 
          epochs=2, batch_size=128, verbose=1, 
          validation_data=(X_train_onehot[-500:], y_train[-500:]))

## Keras lstm

### Sequences

In [166]:
text_seqs_lstm = sequences(serie_texts)
# number of sequences
num_seq=4
# number of features
num_features=4000

#np_features_seq_lstm=np.array()


AttributeError: 'Series' object has no attribute 'text'

In [ ]:

features_seq_lstm=text_seqs_lstm.apply(lambda row: FEATURE_SELECTOR_v4.select_features(serie_texts=row,features_to_calc=features_to_calc,token_params_1=token_params_1,filter_params=filter_params,path_to_features=path_selected_features,flag_extract_features=True,cv_min_df=0,normalization_type=''))
features_seq_lstm

In [ ]:
model = Sequential()

model.add(LSTM(100, input_shape=(num_seq, num_features)))
model.add(Dense(1))

In [ ]:
data = data.reshape(1, num_seq, num_feature)

### Stemming

In [92]:
token_params_keras = set_token_params_1 (get_tokens='text')

In [101]:
stemmed_texts=df_texts.text.apply(lambda row: " ".join(FEATURE_SELECTOR_v4.tokenize(text=row, **token_params_keras)[0]))

In [102]:
stemmed_texts

0        money tell the stori of and is narrat by john ...
1        the open sentenc say the pushcart war start on...
2        dug a mundan is transport in to the magic land...
3        the term gild age commonli given to the era co...
4        the resid of tiamat are split into two clan wi...
5        india opal buloni is a 10 year old girl who ha...
6          to be a jedi is to safeguard peac in the galaxi
7        crippl by a freak accid enterpris ha cross ove...
8        it is the stori of a young woman name much afr...
9        from the hardcov jacket it look as though aust...
10       the plot of permut citi follow the live of sev...
11       the stori narrat by the gigant but docil half ...
12       use altern first person perspect the novel tel...
13       the stori focus on nicol gunther perrin a youn...
14       the first section describ the choos of talia b...
15       versori a literari lectur engag in spread engl...
16       astrid magnussen is a twelv year old girl live.

In [120]:
### Create sequence
vocabulary_size = 20000
tokenizer_stem = Tokenizer(num_words= vocabulary_size)
tokenizer_stem.fit_on_texts(stemmed_texts)
sequences_stem = tokenizer_stem.texts_to_sequences(stemmed_texts)
data_stem = pad_sequences(sequences_stem, maxlen=1200, padding='post')

In [121]:
print(data_stem)

[[ 500   73    1 ...    0    0    0]
 [   1   64 1232 ...    0    0    0]
 [5875    3 2824 ...    0    0    0]
 ...
 [   1  522   42 ...    0    0    0]
 [1647 1992  230 ...    0    0    0]
 [   5 7154   30 ...    0    0    0]]


In [122]:
X_train_lstm_stem, X_test_lstm_stem, y_train_lstm_stem, y_test_lstm_stem = train_test_split(data_stem, df_texts.label, test_size=0.33, random_state=42)

In [125]:
## Network architecture
model = Sequential()
model.add(Embedding(20000, 100, input_length=1200))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
model.fit(X_train_lstm_stem, np.array(y_train_lstm_stem), validation_split=0.4, epochs=5)

Train on 4437 samples, validate on 2959 samples
Epoch 1/5
4437/4437 [==============================] - 313s 71ms/step - loss: 0.6943 - acc: 0.4897 - val_loss: 0.6942 - val_acc: 0.4873
Epoch 2/5
4437/4437 [==============================] - 305s 69ms/step - loss: 0.6936 - acc: 0.4895 - val_loss: 0.6941 - val_acc: 0.4873
Epoch 3/5
4437/4437 [==============================] - 304s 69ms/step - loss: 0.6935 - acc: 0.4904 - val_loss: 0.6937 - val_acc: 0.4873
Epoch 4/5
4437/4437 [==============================] - 305s 69ms/step - loss: 0.6937 - acc: 0.4979 - val_loss: 0.6929 - val_acc: 0.5127
Epoch 5/5
4437/4437 [==============================] - 303s 68ms/step - loss: 0.6934 - acc: 0.5015 - val_loss: 0.6928 - val_acc: 0.5127


### Original text

In [157]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df_texts.text)
sequences = tokenizer.texts_to_sequences(df_texts.text)
data = pad_sequences(sequences, maxlen=1000, padding='post')

In [158]:
X_train_lstm_orig, X_test_lstm_orig, y_train_lstm_orig, y_test_lstm_orig = train_test_split(data, df_texts.label, test_size=0.33, random_state=42)

In [161]:
## Network architecture
model_orig = Sequential()
model_orig.add(Embedding(20000, 200, input_length=1000))
model_orig.add(LSTM(200, dropout=0.5, recurrent_dropout=0.5))
model_orig.add(Dense(1, activation='sigmoid'))
model_orig.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [162]:
model_orig.fit(X_train_lstm_orig, np.array(y_train_lstm_orig), validation_split=0.3, epochs=10)

Train on 5177 samples, validate on 2219 samples
Epoch 1/10
5177/5177 [==============================] - 951s 184ms/step - loss: 0.6752 - acc: 0.5774 - val_loss: 0.6824 - val_acc: 0.4899
Epoch 2/10
5177/5177 [==============================] - 981s 189ms/step - loss: 0.6605 - acc: 0.5926 - val_loss: 0.6615 - val_acc: 0.6048
Epoch 3/10
1856/5177 [=========>....................] - ETA: 9:39 - loss: 0.6360 - acc: 0.6056

KeyboardInterrupt: 

In [138]:
model.fit(X_train_lstm, np.array(y_train_lstm), validation_split=0.4, epochs=10)

Train on 4437 samples, validate on 2959 samples
Epoch 1/40
4437/4437 [==============================] - 311s 70ms/step - loss: 0.4153 - acc: 0.7951 - val_loss: 0.5990 - val_acc: 0.7665
Epoch 2/40
4437/4437 [==============================] - 304s 69ms/step - loss: 0.3529 - acc: 0.8332 - val_loss: 0.6471 - val_acc: 0.7638
Epoch 3/40
4437/4437 [==============================] - 305s 69ms/step - loss: 0.3386 - acc: 0.8425 - val_loss: 0.6215 - val_acc: 0.7682
Epoch 4/40
4437/4437 [==============================] - 316s 71ms/step - loss: 0.3318 - acc: 0.8576 - val_loss: 0.7351 - val_acc: 0.6871
Epoch 5/40
4437/4437 [==============================] - 315s 71ms/step - loss: 0.2976 - acc: 0.8787 - val_loss: 0.6246 - val_acc: 0.7651
Epoch 6/40
1312/4437 [=======>......................] - ETA: 3:13 - loss: 0.2704 - acc: 0.8918

KeyboardInterrupt: 

### PoS 

In [146]:
series_original_word_tokens_grouped_by_texts = df_texts.text.apply(lambda row: nltk.pos_tag(FEATURE_SELECTOR_v4.tokenize(text=row, **token_params_keras)[0]))

In [147]:
series_original_word_tokens_grouped_by_texts

0        [(money, NN), (tell, VB), (the, DT), (stori, N...
1        [(the, DT), (open, JJ), (sentenc, NNS), (say, ...
2        [(dug, VB), (a, DT), (mundan, NN), (is, VBZ), ...
3        [(the, DT), (term, NN), (gild, JJ), (age, NN),...
4        [(the, DT), (resid, NN), (of, IN), (tiamat, NN...
5        [(india, NN), (opal, JJ), (buloni, NN), (is, V...
6        [(to, TO), (be, VB), (a, DT), (jedi, NN), (is,...
7        [(crippl, NN), (by, IN), (a, DT), (freak, JJ),...
8        [(it, PRP), (is, VBZ), (the, DT), (stori, NN),...
9        [(from, IN), (the, DT), (hardcov, NN), (jacket...
10       [(the, DT), (plot, NN), (of, IN), (permut, NN)...
11       [(the, DT), (stori, JJ), (narrat, NN), (by, IN...
12       [(use, NN), (altern, JJ), (first, JJ), (person...
13       [(the, DT), (stori, JJ), (focus, NN), (on, IN)...
14       [(the, DT), (first, JJ), (section, NN), (descr...
15       [(versori, FW), (a, DT), (literari, NN), (lect...
16       [(astrid, JJ), (magnussen, NN), (is, VBZ), (a,.

In [148]:
pos_texts=series_original_word_tokens_grouped_by_texts.apply(lambda row: " ".join([w[1] for w in row]))

In [149]:
pos_texts

0        NN VB DT NN IN CC VBZ VBN IN NN PRP DT NN NN I...
1        DT JJ NNS VBP DT JJ NN NN IN DT NN IN NN CD CD...
2        VB DT NN VBZ RB IN TO DT JJ NN IN NN WRB PRP V...
3         DT NN JJ NN NN VBN TO DT NN NN IN DT NN IN NN NN
4        DT NN IN NN VBP VBN IN CD JJ NN WP VBZ NN NN C...
5        NN JJ NN VBZ DT CD NN JJ NN WP VBZ RB VB TO DT...
6                        TO VB DT NN VBZ TO VB NN IN DT NN
7        NN IN DT JJ NN NN NN NN IN IN DT JJ NNS CC IN ...
8        PRP VBZ DT NN IN DT JJ NN NN JJ NN CC PRP$ NN ...
9             IN DT NN NN PRP VBD IN IN NN MD VB VBN IN NN
10       DT NN IN NN NN VBP DT NN IN NN NN IN DT JJ NN ...
11       DT JJ NN IN DT NN CC JJ NN JJ JJ NN JJ JJ NN I...
12       NN JJ JJ NN VBP DT NN VBP DT NN IN NN NN JJ CD...
13       DT JJ NN IN NN NN VBD DT JJ NN IN JJ JJ NN NN ...
14       DT JJ NN VBZ DT NN IN NN IN DT NN NN CC VB NN ...
15       FW DT NN NN NN IN JJ JJ NN IN NN TO VB NN IN N...
16                JJ NN VBZ DT JJ NN JJ NN VBP IN JJ NN 

In [150]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(pos_texts)
sequences = tokenizer.texts_to_sequences(pos_texts)
pos_data = pad_sequences(sequences, maxlen=800, padding='post')
pos_data

array([[ 1,  5,  3, ...,  0,  0,  0],
       [ 3,  4, 15, ...,  0,  0,  0],
       [ 5,  3,  1, ...,  0,  0,  0],
       ...,
       [ 3,  1,  4, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0],
       [ 1,  1,  6, ...,  0,  0,  0]])

In [153]:
X_train_lstm_pos, X_test_lstm_pos, y_train_lstm_pos, y_test_lstm_pos = train_test_split(pos_data, df_texts.label, test_size=0.33, random_state=42)

In [154]:
## Network architecture
model_pos = Sequential()
model_pos.add(Embedding(20000, 200, input_length=800))
model_pos.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_pos.add(Dense(1, activation='sigmoid'))
model_pos.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [155]:
model_pos.fit(X_train_lstm_pos, np.array(y_train_lstm_pos), validation_split=0.2, epochs=20)

Train on 5916 samples, validate on 1480 samples
Epoch 1/20
5916/5916 [==============================] - 401s 68ms/step - loss: 0.6938 - acc: 0.5145 - val_loss: 0.6932 - val_acc: 0.5027
Epoch 2/20
5916/5916 [==============================] - 389s 66ms/step - loss: 0.6938 - acc: 0.4909 - val_loss: 0.6934 - val_acc: 0.4973
Epoch 3/20
5916/5916 [==============================] - 389s 66ms/step - loss: 0.6937 - acc: 0.5041 - val_loss: 0.6932 - val_acc: 0.4973
Epoch 4/20
5916/5916 [==============================] - 388s 66ms/step - loss: 0.6936 - acc: 0.4949 - val_loss: 0.6931 - val_acc: 0.5027
Epoch 5/20
5916/5916 [==============================] - 389s 66ms/step - loss: 0.6936 - acc: 0.4912 - val_loss: 0.6931 - val_acc: 0.4973
Epoch 6/20
5916/5916 [==============================] - 390s 66ms/step - loss: 0.6932 - acc: 0.5056 - val_loss: 0.6933 - val_acc: 0.4973
Epoch 7/20
5916/5916 [==============================] - 389s 66ms/step - loss: 0.6934 - acc: 0.4968 - val_loss: 0.6932 - val_acc: 